# Zillow Scrapper
## Julie Huang

In [2]:
## Import beautiful soup to scrape data from Zillow
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [3]:
#Base class for property
class Property(object):
    """A property listed on Zillow


    Attributes:
        address: The address of the property as a string.
        bed: An integer representing the number of bedrooms the property has.
        bath: A float representing the no. of bathrooms the property has
        size: An interger representing the sqft of the property
        price: An interger representing the price of the property
    """


    def __init__(self):
        self.address = ""
        self.bed = 0.0
        self.bath = 0.0
        self.size = 0.0
        self.price = 0.0
        
    def price_per_sqft(self):
        """Return the sale price for this vehicle as a float amount."""
        if self.size != 0.0:
            pps = self.price/(self.size)
        else:
            pps = "Empty"
        return pps
    
    def price_per_bed(self):
        if self.bed != 0.0:
            ppb = self.price/self.bed
        else:
            ppb = "Empty"
        """Return the sale price for this vehicle as a float amount."""
        return ppb

In [64]:
# Find the html class for "Next Page"
for item in soup.find_all("li", {'class':'zsg-pagination-next'}):
    print item
#https://www.zillow.com/homes/02148_rb/
#https://www.zillow.com/homes/02148_rb/2_p/
#<li class="zsg-pagination-next"><a class="off" href="/homes/02148_rb/2_p/" onclick="SearchMain.changePage(2);return false;">Next</a></li>

<li class="zsg-pagination-next"><a class="off" href="/homes/02148_rb/2_p/" onclick="SearchMain.changePage(2);return false;">Next</a></li>


In [134]:
## How urls look like when there are multiple pages to list all properties
zipcode = raw_input("Type a zipcode ")
i = 1
url = "https://www.zillow.com/homes/"+zipcode +"_rb/"+ str(i) +'_p/'
print url
r = requests.get(url)
soup = BeautifulSoup(r.content) #make a soup for each url

Type a zipcode 02148
https://www.zillow.com/homes/02148_rb/1_p/


#print soup.prettify()

for item in soup.find_all("div", {'class':'zsg-photo-card-caption'}):
    #print item.contents[0].text #Apartment For Rent
    #print item.contents[1].text #1$1,585+2$2,145+ #$239,9002 bds · 1 ba · 738 sqft
    #print item.contents[2].text #18 hours ago·20 Park View Ter # 000, Malden, MA
    print item.contents[1].text #$1,500/mo
    #print item.contents[1].find_all("span", {'class':'interpunct'}) #1 bd · 1 ba · -- sqft

In [34]:
# The first method to save the data, by using a dictionary

Zillow = {}
id = 0
for item in soup.find_all("div", {'class':'zsg-photo-card-caption'}):
#    if item.get('class')==['zsg-photo-card-caption']:
        P = Property()
        for i in item.find_all("span"):
            if i.get('class')==['zsg-photo-card-status']:
                P.status = i.text
            if i.get('class')==['zsg-photo-card-price']:
                P.price = i.text
                P.price = float(i.text.replace("$","").replace("/mo","").replace(",","").replace("+",""))
                if P.price == 0.0:
                    P.price = NA
            else: 
                pass
            if i.get('class')==['zsg-photo-card-info'] and P.price: #P.price needs to exist to proceed on info and address
                P.info = i.text.replace("Studio","0.5 bd") #info needs to be sliced, here Studio is converted to 0.5 bedroom
                #print P.info
                P.bed = float(P.info.split()[0]) 
                P.bath = float(P.info.split()[3])
                P.size = float(P.info.split()[6].replace("--","0").replace("sqft","0").replace(",","").replace("+","")) #This number can be "0"
            if i.get('class')==['zsg-photo-card-address'] and P.price:
                P.address = i.text  
        print P.status+" (status)\n" + str(P.price)+" (price)\n"+ str(P.bed)+" (beds #0.5 for Studio)\n"+ str(P.bath)+" (baths)\n"+ str(P.size)+" (size)\n"+str(P.address) +" (address)\n"+str(P.price_per_sqft())+" (price_per_sqft)\n\n\n"
        Zillow.update({id:P})
        id+=1
        
##info 1$1,585+2$2,145+
##adress 500 Broadway, Malden, MA

####We do not want it when price is empty

##price $1,700/mo
##info 2 bds · 1 ba · -- sqft
##adress Newman Rd, Malden, MA

####This is the type we want

Apartment For Rent (status)
1850.0 (price)
2.0 (beds #0.5 for Studio)
1.0 (baths)
0.0 (size)
Pleasant St, Malden, MA (address)
Empty (price_per_sqft)



Apartment For Rent (status)
2700.0 (price)
4.0 (beds #0.5 for Studio)
1.0 (baths)
0.0 (size)
(Undisclosed Address), Malden, MA (address)
Empty (price_per_sqft)



Pre-Foreclosure (status)
0.0 (price)
0.0 (beds #0.5 for Studio)
0.0 (baths)
0.0 (size)
 (address)
Empty (price_per_sqft)



Foreclosed (status)
0.0 (price)
0.0 (beds #0.5 for Studio)
0.0 (baths)
0.0 (size)
 (address)
Empty (price_per_sqft)



Pre-Foreclosure (status)
0.0 (price)
0.0 (beds #0.5 for Studio)
0.0 (baths)
0.0 (size)
 (address)
Empty (price_per_sqft)



Apartment For Rent (status)
2300.0 (price)
2.0 (beds #0.5 for Studio)
2.0 (baths)
0.0 (size)
72 Ashland St, Malden, MA (address)
Empty (price_per_sqft)



Foreclosed (status)
0.0 (price)
0.0 (beds #0.5 for Studio)
0.0 (baths)
0.0 (size)
 (address)
Empty (price_per_sqft)



Pre-Foreclosure (status)
0.0 (price)
0.0 (b

In [44]:
for i in range(0,id):
    print str(Zillow[i].price) + '   ' + str(Zillow[i].bed) + '   '  + str(Zillow[i].size)+ '   ' + Zillow[i].address

3300.0   1.0   0.0   170 Bleecker St APT 5N, New York, NY
599000.0   1.0   0.0   101 Thompson St APT 20, New York, NY
3300.0   1.0   0.0   31 Prince St # 3B, New York, NY
2550.0   0.5   0.0   160 Prince St APT 10, New York, NY
10000.0   1.0   900.0   262 Mott St APT 512, New York, NY
4095.0   1.0   0.0   301 Elizabeth St # 1020, New York, NY
0.0   0.0   0.0   
4000.0   1.0   550.0   184 Prince St APT 2, New York, NY
2950.0   1.0   500.0   236 Mulberry St APT 15, New York, NY
3300.0   0.5   0.0   155 Bleecker St # 1JW, New York, NY
2950.0   0.5   1300.0   24 Minetta Ln APT 3R, New York, NY
0.0   0.0   0.0   
0.0   0.0   0.0   
2895.0   1.0   0.0   240 Sullivan St APT 8, New York, NY
11500.0   3.0   0.0   96 Macdougal St # 2, New York, NY
2775.0   1.0   0.0   174 Spring St APT 21, New York, NY
4200000.0   2.0   1808.0   109 Greene St APT 5B, New York, NY
0.0   0.0   0.0   
7995000.0   2.0   2660.0   285 Lafayette St # 5E, New York, NY
0.0   0.0   0.0   
3400.0   1.0   0.0   214 Mulberry 

In [35]:
# The second method to save the data, by using a list
property_list=[]
prop_info = soup.find_all("div",{"class":"zsg-photo-card-caption"})
for item in prop_info:
    for i in item.find_all("span"):
        if i.get('class')==['zsg-photo-card-status']:
            try:
                status = i.text
            except:
                status = 'NA'
        if i.get('class')==['zsg-photo-card-price']:
            try:
                price = float(i.text.replace("$","").replace("/mo","").replace(",","").replace("+",""))
            except:
                price = 0.0
        if i.get('class')==['zsg-photo-card-info']:
            info = i.text.replace("Studio","0.5 bd")
            try:
                bed = float(P.info.split()[0]) 
            except:
                bed = 0.0
            try:
                bath = float(P.info.split()[3])
            except:
                bath = 0.0
            try:
                size = float(P.info.split()[6].replace("--","0").replace("sqft","0").replace(",","").replace("+",""))
            except:
                size = 0.0
        if i.get('class')==['zsg-photo-card-address']:
            try:
                address = i.text
            except:
                address = ''
    prop=[status,price,bed,bath,size,address]
    property_list.append(prop)

In [37]:
len(property_list)

25